In [1]:
import os
os.chdir('..')
import shutil

In [2]:
from detectron2_1.AL.AL_select import *

In [3]:
import pickle
import json
import numpy as np
import torch
from tqdm import tqdm

## Score aggregation

In [4]:
def score_agg(results):
    '''
    Aggregate score for each image by taking average
    '''
    file_name_list = []
    score_list = []

    for i, res in tqdm(enumerate(results)):
        if res['file_name'] not in file_name_list: # see a brand new image
            if i != 0: # append previous images' score
                score_list.append(new_score)
            file_name_list.append(res['file_name']) # append new file name
            new_score = res['score_al'] # update score
            new_ct = 1 # restart box count 

        else:
            new_score = (res['score_al'] + new_score*new_ct) / (new_ct + 1) # update score (average)
            new_ct += 1 # update box count 

    score_list.append(new_score) # append last images' score
    return file_name_list, score_list

In [9]:
with open('output/feature_emb.pkl', 'rb') as handle:
    feature_emb = pickle.load(handle)
    
with open('output/entropy_al.json', 'r') as handle:
    entropy = json.load(handle)
    
with open('output/margin_al.json', 'r') as handle:
    margin = json.load(handle)
    
with open('output/perturb_al.json', 'r') as handle:
    perturb = json.load(handle)

In [10]:
feature_embeddings = np.asarray([x['feature'] for x in feature_emb])

feature_files = np.asarray([x['file_name'] for x in feature_emb])

In [ ]:
file_name_list, entropy_scores = score_agg(entropy)

666027it [06:26, 807.02it/s]  

In [7]:
len(entropy)

765882

In [8]:
file_name_list, margin_scores = score_agg(margin)

765882it [10:59, 1161.09it/s] 


In [ ]:
file_name_list, perturb_scores = score_agg(perturb)

## Select

In [11]:
## some images may not have predictions, simply ignore
existance = list(map(lambda x: x in file_name_list, feature_files))

## images without any predicted box
mask = np.ones((len(feature_files),), bool)
mask[np.where(np.asarray(existance) == False)[0]] = False # mask of shape (n,)

## remove from feature embedddings
feature_emb_select = feature_embeddings[mask,:]

assert len(feature_emb_select) == len(margin_scores)

In [12]:
print(len(feature_emb_select))
print(len(margin_scores))

49030
49030


In [ ]:
c_sets = kmeans_plus(S=entropy_scores, feat=feature_emb, N=5000)
np.save('./detector_results/entropy_select5000_kmeans++.npy', c_sets)

In [ ]:
c_sets = core_set(S=margin_scores, feat=feature_emb_select, N=5000)
np.save('./detector_results/margin_select5000_coreset.npy', c_sets)

In [ ]:
c_sets = kmeans_plus(S=margin_scores, feat=feature_emb_select, N=5000)
np.save('./detector_results/margin_select5000_kmeans++.npy', c_sets)